In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_preprocess

# ——— your folders ———
TRAIN_DIR = r'D:\iot project\train'
VAL_DIR   = r'D:\iot project\val'
IMG_SIZE  = (96, 96)
BATCH     = 32
NUM_CLASSES = 3  # cow, camel, goat

# 1. Datasets
train_ds = image_dataset_from_directory(
    TRAIN_DIR,
    label_mode='int',
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=True,
    seed=42
).map(lambda x, y: (mobilenet_preprocess(x), y)) \
 .prefetch(tf.data.AUTOTUNE)

val_ds = image_dataset_from_directory(
    VAL_DIR,
    label_mode='int',
    image_size=IMG_SIZE,
    batch_size=BATCH,
    shuffle=False
).map(lambda x, y: (mobilenet_preprocess(x), y)) \
 .prefetch(tf.data.AUTOTUNE)

# 2. Build MobileNetV1-based model
base = tf.keras.applications.MobileNet(
    input_shape=(*IMG_SIZE, 3),
    alpha=0.25,
    weights='imagenet',     # transfer learn from ImageNet
    include_top=False
)
base.trainable = False     # freeze for initial training

model = models.Sequential([
    layers.Input(shape=(*IMG_SIZE, 3)),
    base,
    layers.GlobalAveragePooling2D(),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax'),
])

# 3. Compile & train head
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

# 4. Unfreeze some of the base for fine-tuning
base.trainable = True
for layer in base.layers[:50]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# 5. Optional: Combine histories or save model
model.save('mobilenetv1_finetuned.h5')
print("✅ Done — MobileNetV1 training finished.")


Found 719 files belonging to 3 classes.
Found 181 files belonging to 3 classes.


C:\Users\white\AppData\Local\Temp\ipykernel_16568\751258837.py:34: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base = tf.keras.applications.MobileNet(


Epoch 1/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 16s 360ms/step - accuracy: 0.4691 - loss: 1.9329 - val_accuracy: 0.8619 - val_loss: 0.3936
Epoch 2/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.7865 - loss: 0.5552 - val_accuracy: 0.8950 - val_loss: 0.2643
Epoch 3/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 99ms/step - accuracy: 0.8602 - loss: 0.3582 - val_accuracy: 0.9061 - val_loss: 0.2199
Epoch 4/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 114ms/step - accuracy: 0.8940 - loss: 0.2856 - val_accuracy: 0.9116 - val_loss: 0.2033
Epoch 5/5
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 94ms/step - accuracy: 0.9106 - loss: 0.2411 - val_accuracy: 0.9227 - val_loss: 0.1813
Epoch 1/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 19s 195ms/step - accuracy: 0.7624 - loss: 0.6749 - val_accuracy: 0.9337 - val_loss: 0.1718
Epoch 2/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 148ms/step - accuracy: 0.8744 - loss: 0.3629 - val_accuracy: 0.9448 - val_loss: 0.1687
Epoch 3/10
23/23 ━━━━━━━━━━━━━━━━━━━━ 5s 118ms/step - accuracy: 0.8676 - loss: 0.3441 - val_accuracy: 0.9392 -

✅ Done — MobileNetV1 training finished.


In [4]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model(r'D:\iot project\mobilenetv1_finetuned.h5', compile=False)

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

def rep_data_gen():
    for _ in range(100):
        # random float in [0,1], matching your model's float32 input
        dummy = np.random.random_sample((1,96,96,3)).astype(np.float32)
        yield [dummy]

converter.representative_dataset = rep_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]

# Keep the I/O as float32 so it matches your model signature
# (the weights & activations will still be quantized to int8 internally)
converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32

tflite_model = converter.convert()
with open('mobilenetv1_quant.tflite', 'wb') as f:
    f.write(tflite_model)

print("✅ Quantization complete, saved mobilenetv1_quant.tflite")


INFO:tensorflow:Assets written to: C:\Users\white\AppData\Local\Temp\tmpdo3thepj\assets


INFO:tensorflow:Assets written to: C:\Users\white\AppData\Local\Temp\tmpdo3thepj\assets


Saved artifact at 'C:\Users\white\AppData\Local\Temp\tmpdo3thepj'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 96, 96, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  1869806412880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1869806413840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1869806414224: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1869806414032: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1869806412496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1869806415376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1869806415760: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1869806416144: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1869806415952: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1869806413072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  186980

c:\Users\white\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\lite\python\convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


✅ Quantization complete, saved mobilenetv1_quant.tflite


In [ ]:
import tensorflow as tf
import numpy as np

import os

# ——— EDIT THESE PATHS IF NEEDED ———
TFLITE_MODEL_PATH = r'D:\iot project\mobilenetv1_quant.tflite'
VAL_DIR            = r'D:\iot project\val'     # cow/, camel/, goat/
BATCH_SIZE         = 32
IMG_SIZE           = (96, 96)

# 1. Load validation dataset (values in [0,255], dtype uint8)
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    VAL_DIR,
    labels='inferred',
    label_mode='int',
    batch_size=BATCH_SIZE,
    image_size=IMG_SIZE,
    shuffle=False
)

# 2. Load TFLite interpreter
interpreter = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)
interpreter.allocate_tensors()
input_details  = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

print("→ Model input details:", input_details)
print("→ Model expects dtype:", input_details['dtype'],
      "shape:", input_details['shape'])

# 3. Evaluate
total, correct = 0, 0

for batch_images, batch_labels in val_ds:
    # Convert to numpy for feeding the interpreter
    imgs = batch_images.numpy()           # shape (batch,128,128,3), uint8

    # 3a. Preprocess depending on expected input dtype
    if input_details['dtype'] == np.uint8:
        # Model wants uint8 [0..255]
        batch_input = imgs.astype(np.uint8)

    elif input_details['dtype'] == np.float32:
        # Model wants float32 in [0..1]
        batch_input = (imgs / 255.0).astype(np.float32)

    else:
        raise ValueError(f"Unsupported input dtype: {input_details['dtype']}")

    # 3b. Run per-image inference
    for img, true_label in zip(batch_input, batch_labels.numpy()):
        # Expand dims to match [1, h, w, c]
        interpreter.set_tensor(input_details['index'], np.expand_dims(img, 0))
        interpreter.invoke()
        output = interpreter.get_tensor(output_details['index'])[0]  # e.g. shape [3]
        pred_label = np.argmax(output)

        if pred_label == true_label:
            correct += 1
        total += 1

# 4. Report
acc = 100 * correct / total if total else 0
print(f'Validation accuracy: {acc:.2f}%  ({correct}/{total})')


Found 181 files belonging to 3 classes.
→ Model input details: {'name': 'serving_default_input_layer_1:0', 'index': 0, 'shape': array([ 1, 96, 96,  3], dtype=int32), 'shape_signature': array([-1, 96, 96,  3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
→ Model expects dtype: <class 'numpy.float32'> shape: [ 1 96 96  3]


c:\Users\white\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\lite\python\interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


Validation accuracy: 82.87%  (150/181)
